In [2]:
import pandas as pd

# Replace 'your_file.parquet' with the path to your Parquet file
df = pd.read_parquet('../data_cleaned/parquet_outputs/Timo/output-all.parquet')

# Display the first few rows
df.head()

,experiment_id,base,caffeine_ml,source,time,accelerometer_x,accelerometer_y,accelerometer_z,gyroscope_x,gyroscope_y,gyroscope_z,linear_acceleration_x,linear_acceleration_y,linear_acceleration_z
0,2025-06-11,0,230,Timo,1.026389,-0.047258,-0.048156,9.935597,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-06-11,0,230,Timo,1.028774,-0.023330,-0.050548,9.835099,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-06-11,0,230,Timo,1.040701,-0.013759,0.033200,9.837492,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-06-11,0,230,Timo,1.043086,0.026919,0.037986,9.866206,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-06-11,0,230,Timo,1.045471,0.079561,-0.012263,9.918848,NaN,NaN,NaN,NaN,NaN,NaN


### Interactive Python Code Snippet


In [13]:
from ipywidgets import widgets, interactive_output, HBox, VBox
import pandas as pd

# --- Widgets ---
date_widget = widgets.Dropdown(
    options=['All'] + sorted(df['experiment_id'].unique()),
    description='Date:'
)
base_widget = widgets.Dropdown(
    options=['All'] + sorted(df['base'].unique()),
    description='Base:'
)
sensor_widget = widgets.SelectMultiple(
    options=[c for c in df.columns if c.endswith(('_x','_y','_z'))],
    description='Sensors:'
)
time_min, time_max = int(df['time'].min()), int(df['time'].max())
time_widget = widgets.IntRangeSlider(
    value=(time_min, time_max),
    min=time_min, max=time_max, step=1,
    description='Time:',
    continuous_update=False,
    layout=widgets.Layout(width='200px')
)

# NEW: row‐window as two numeric inputs
n = len(df)
start_row_widget = widgets.BoundedIntText(
    value=1, min=1, max=n, step=1,
    description='Start row:'
)
end_row_widget = widgets.BoundedIntText(
    value=5, min=1, max=n, step=1,
    description='End row:'
)

# --- Filtering function ---
def filter_dataframe(date, base, sensors, time_range, start_row, end_row):
    filtered = df.copy()
    # date/base
    if date != 'All':
        filtered = filtered[filtered['experiment_id'] == date]
    if base != 'All':
        filtered = filtered[filtered['base'] == base]
    # time
    t0, t1 = time_range
    filtered = filtered[(filtered['time'] >= t0) & (filtered['time'] <= t1)]
    # sensors
    if sensors:
        cols = ['experiment_id','base','caffeine_ml','source','time'] + list(sensors)
        filtered = filtered[cols]
    # clip start/end
    start = max(1, min(start_row, len(filtered)))
    end   = max(start, min(end_row,   len(filtered)))
    display(filtered.iloc[start-1:end])

# --- Wire up ---
out = interactive_output(filter_dataframe, {
    'date':       date_widget,
    'base':       base_widget,
    'sensors':    sensor_widget,
    'time_range': time_widget,
    'start_row':  start_row_widget,
    'end_row':    end_row_widget,
})

# Layout
controls = HBox([
    date_widget, base_widget, sensor_widget,
    time_widget, start_row_widget, end_row_widget
])
display(VBox([controls, out]))
